In [4]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tslearn.clustering import TimeSeriesKMeans
from tslearn.preprocessing import TimeSeriesScalerMinMax
from tslearn.preprocessing import TimeSeriesScalerMinMax
import matplotlib.pyplot as plt
import joblib
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense, Bidirectional
from keras.regularizers import l2

In [15]:
file_path= "C:/Users/vgarciario/desktop/UA MASTER/labo3/final_dataset_descr.csv"
df1 = pd.read_csv(file_path, sep='\t')

file_path2= "C:/Users/vgarciario/desktop/UA MASTER/labo3/productos_a_predecir.txt"
pp= pd.read_csv(file_path2, sep='\t')

In [16]:
df = pd.merge(df1, pp, on='product_id', how='inner')

In [5]:
df.columns

Index(['periodo', 'customer_id', 'product_id', 'plan_precios_cuidados',
       'cust_request_qty', 'cust_request_tn', 'tn', 'cat1', 'cat2', 'cat3',
       'brand', 'sku_size', 'descripcion', 'quarter', 'month', 'close_quarter',
       'age'],
      dtype='object')

In [7]:
# df.drop(columns=['plan_precios_cuidados', 'cust_request_qty','cust_request_tn','quarter', 'month', 'close_quarter', 'age'], inplace=True)

In [6]:
df.columns

Index(['periodo', 'customer_id', 'product_id', 'plan_precios_cuidados',
       'cust_request_qty', 'cust_request_tn', 'tn', 'cat1', 'cat2', 'cat3',
       'brand', 'sku_size', 'descripcion', 'quarter', 'month', 'close_quarter',
       'age'],
      dtype='object')

In [17]:
#Paso1: reemplazar 082019 por promedio 07 y 09
df['periodo'] = df['periodo'].astype(str).str.strip()

# Filtrar los datos por los periodos 201907, 201908 y 201909
df_filtered = df[df['periodo'].isin(['201907', '201908', '201909'])]

# # Pivotear los datos para tener columnas separadas para cada periodo
pivoted_sales = df_filtered.pivot_table(index=['product_id', 'customer_id'], columns='periodo', values='tn').reset_index()

# # Asegurar que las columnas 201907 y 201909 existen en el DataFrame
pivoted_sales = pivoted_sales.reindex(columns=['product_id', 'customer_id', '201907', '201908', '201909'])

# # Calcular el promedio de julio y septiembre
pivoted_sales['201908'] = pivoted_sales[['201907', '201909']].mean(axis=1)

# # Convertir de nuevo al formato largo
updated_sales = pivoted_sales.melt(id_vars=['product_id', 'customer_id'], value_vars=['201907', '201908', '201909'], 
                                   var_name='periodo', value_name='tn')

# # Unir con el dataframe original
df.set_index(['product_id', 'customer_id', 'periodo'], inplace=True)
df.update(updated_sales.set_index(['product_id', 'customer_id', 'periodo']))
df.reset_index(inplace=True)

display(df)

,product_id,customer_id,periodo,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn,cat1,cat2,cat3,brand,sku_size,descripcion,quarter,month,close_quarter,age
0,20001,10001,201701,0,11,99.43861,99.43861,HC,ROPA LAVADO,Liquido,ARIEL,3000,genoma,Q1,1,0,0
1,20001,10002,201701,0,17,38.68301,35.72806,HC,ROPA LAVADO,Liquido,ARIEL,3000,genoma,Q1,1,0,0
2,20001,10003,201701,0,17,143.49426,143.49426,HC,ROPA LAVADO,Liquido,ARIEL,3000,genoma,Q1,1,0,0
3,20001,10004,201701,0,9,184.72927,184.72927,HC,ROPA LAVADO,Liquido,ARIEL,3000,genoma,Q1,1,0,0
4,20001,10005,201701,0,23,19.08407,19.08407,HC,ROPA LAVADO,Liquido,ARIEL,3000,genoma,Q1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2293476,21214,10430,201912,0,1,0.00179,0.00179,PC,DEOS,RollOn,NIVEA,50,Aroma 14,Q4,12,1,3
2293477,21214,10441,201912,0,1,0.00239,0.00239,PC,DEOS,RollOn,NIVEA,50,Aroma 14,Q4,12,1,3
2293478,21214,10448,201912,0,1,0.00299,0.00299,PC,DEOS,RollOn,NIVEA,50,Aroma 14,Q4,12,1,3
2293479,21214,10505,201912,0,1,0.00179,0.00179,PC,DEOS,RollOn,NIVEA,50,Aroma 14,Q4,12,1,3


In [18]:
# Filtrar el DataFrame para los meses y IDs específicos
df_subset = df[(df['product_id'] == 20001) & (df['customer_id'] == 10001)]

# Imprimir el resultado
print(df_subset[['product_id', 'customer_id', 'periodo', 'tn']])
df_subset.tail(10)

      product_id  customer_id periodo         tn
0          20001        10001  201701   99.43861
186        20001        10001  201702  198.84365
371        20001        10001  201703   92.46537
559        20001        10001  201704   13.29728
663        20001        10001  201705  101.00563
901        20001        10001  201706  128.04792
1121       20001        10001  201707  101.20711
1272       20001        10001  201708   43.33930
1508       20001        10001  201709  289.35024
1671       20001        10001  201710  222.11389
1767       20001        10001  201711  111.54944
1942       20001        10001  201712  131.27150
2134       20001        10001  201801   49.61857
2286       20001        10001  201802   88.44065
2443       20001        10001  201803  214.72336
2647       20001        10001  201804  132.83419
2801       20001        10001  201805  165.73507
2989       20001        10001  201806   92.29410
3177       20001        10001  201807  256.05760
3344       20001    

,product_id,customer_id,periodo,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn,cat1,cat2,cat3,brand,sku_size,descripcion,quarter,month,close_quarter,age
4530,20001,10001,201903,0,23,142.87158,130.54927,HC,ROPA LAVADO,Liquido,ARIEL,3000,genoma,Q1,3,1,26
4721,20001,10001,201904,0,33,364.37071,364.37071,HC,ROPA LAVADO,Liquido,ARIEL,3000,genoma,Q2,4,0,27
4898,20001,10001,201905,0,31,439.90647,439.90647,HC,ROPA LAVADO,Liquido,ARIEL,3000,genoma,Q2,5,0,28
5020,20001,10001,201906,0,7,65.92436,65.92436,HC,ROPA LAVADO,Liquido,ARIEL,3000,genoma,Q2,6,1,29
5202,20001,10001,201907,0,14,144.78714,144.78714,HC,ROPA LAVADO,Liquido,ARIEL,3000,genoma,Q3,7,0,30
5383,20001,10001,201908,0,9,33.63991,126.91979,HC,ROPA LAVADO,Liquido,ARIEL,3000,genoma,Q3,8,0,31
5546,20001,10001,201909,0,18,111.51691,109.05244,HC,ROPA LAVADO,Liquido,ARIEL,3000,genoma,Q3,9,1,32
5712,20001,10001,201910,0,21,178.49426,176.02980,HC,ROPA LAVADO,Liquido,ARIEL,3000,genoma,Q4,10,0,33
5840,20001,10001,201911,0,21,240.59870,236.65556,HC,ROPA LAVADO,Liquido,ARIEL,3000,genoma,Q4,11,0,34
5996,20001,10001,201912,0,18,214.72185,180.21938,HC,ROPA LAVADO,Liquido,ARIEL,3000,genoma,Q4,12,1,35


no corri lo training threshold

In [9]:
training_threshold = 12

# Contar el número de registros por product_id, customer_id y periodo
product_history = df.groupby(['product_id', 'customer_id']).size().reset_index(name='counts')

# Filtrar productos por customer_id con menos registros que el threshold
products_to_keep = product_history[product_history['counts'] >= training_threshold][['product_id', 'customer_id']].drop_duplicates()
df_filtered = df.merge(products_to_keep, on=['product_id', 'customer_id'], how='inner')

In [19]:

# Paso 3: Aplicar LabelEncoder a las columnas categoricas
categorical_cols = ['cat1', 'cat2', 'cat3', 'brand', 'descripcion']
label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le



In [20]:
# Paso 4: Agrupar las ventas por periodo, cat1, cat2, cat3, marca y descripcion

# Supongamos que 'periodo' es una columna de tipo string con formato 'YYYYMM'
df['periodo'] = pd.to_datetime(df['periodo'], format='%Y%m', errors='coerce')

# Filtrar datos para diciembre de 2019
df_diciembre_2019 = df[(df['periodo'].dt.year == 2019) & (df['periodo'].dt.month == 12)]

# Agrupar las ventas del año 2019 por periodo, cat1, cat2, cat3, brand, descripcion y product_id
grouped_sales_2019 = df_diciembre_2019.groupby(['cat1', 'cat2', 'cat3', 'brand','product_id'])['tn'].sum().reset_index()
#'periodo', 'cat1', 'cat2', 'cat3', 'brand', 'descripcion', 

# Calcular el promedio de ventas por periodo para el año 2019 por grupo de cat1, cat2, cat3, brand, descripcion y product_id
#average_sales_2019 = grouped_sales_2019.groupby(['cat1', 'cat2', 'cat3', 'brand', 'descripcion', 'product_id'])['tn'].mean().reset_index()

# Calcular el promedio total de ventas por grupo de cat1, cat2, cat3, brand y descripcion para el año 2019
group_totals_2019 = df_diciembre_2019.groupby(['cat1', 'cat2', 'cat3', 'brand'])['tn'].sum().reset_index()

# Unir para calcular el ratio
ratios_2019 = pd.merge(grouped_sales_2019, group_totals_2019, on=['cat1', 'cat2', 'cat3', 'brand'], suffixes=('', '_total'))
ratios_2019['ratio'] = ratios_2019['tn'] / ratios_2019['tn_total']

# Crear un diccionario de ratios
ratio_dict = ratios_2019.set_index(['cat1', 'cat2', 'cat3', 'brand', 'product_id'])['ratio'].to_dict()


In [21]:
# Filtrar el DataFrame para los meses y IDs específicos
df_subset = ratios_2019[(ratios_2019['product_id'] == 20001)]

# Imprimir el resultado
print(df_subset)
df_subset.tail(10)

     cat1  cat2  cat3  brand  product_id          tn    tn_total     ratio
242     1    10    47      0       20001  1504.68856  1892.55454  0.795057


,cat1,cat2,cat3,brand,product_id,tn,tn_total,ratio
242,1,10,47,0,20001,1504.68856,1892.55454,0.795057


In [22]:

# Paso 5: Agrupar las ventas por periodo, cat1, cat2, cat3, marca y customer_id
grouped_df = df.groupby(['periodo', 'cat1', 'cat2', 'cat3', 'brand', 'customer_id']).agg({
    'cust_request_qty': 'sum',
    'cust_request_tn': 'sum',
    'tn': 'sum'
}).reset_index()

# Crear un diccionario para almacenar los scalers
scalers = {}
scaled_df = grouped_df.copy()

# Aplicar StandardScaler a cada columna de interés
for col in ['cust_request_qty', 'cust_request_tn', 'tn']:
    scaler = StandardScaler()
    scaled_df[col] = scaler.fit_transform(scaled_df[[col]])
    scalers[col] = scaler

# Guardar los scalers para su uso posterior
joblib.dump(scalers, 'scalers.pkl')


['scalers.pkl']

In [23]:
scaled_df.head()

,periodo,cat1,cat2,cat3,brand,customer_id,cust_request_qty,cust_request_tn,tn
0,2017-01-01,0,0,4,22,10001,-0.117996,-0.132442,-0.133144
1,2017-01-01,0,0,4,22,10002,0.787751,-0.094268,-0.093927
2,2017-01-01,0,0,4,22,10003,-0.072708,-0.116217,-0.116476
3,2017-01-01,0,0,4,22,10004,0.153728,-0.114309,-0.114516
4,2017-01-01,0,0,4,22,10005,-0.299145,-0.159164,-0.160596


In [27]:
# Asumiendo que scaled_df ya está cargado y escalado
scaled_df['periodo'] = pd.to_datetime(scaled_df['periodo'], format='%Y%m')

# Función para crear secuencias de tiempo
def crear_secuencias(data, n_steps, step_ahead=1):
    X, y = [], []
    for i in range(len(data) - n_steps - step_ahead):
        X.append(data[['tn']].iloc[i:i+n_steps].values)
        y.append(data['tn'].iloc[i+n_steps+step_ahead-1])
    return np.array(X), np.array(y)

# Función para construir el modelo LSTM bidireccional
def build_lstm_model(input_shape):
    model = Sequential()
    model.add(Bidirectional(LSTM(128, activation='tanh', kernel_regularizer=l2(0.7), return_sequences=True), input_shape=input_shape))
    model.add(Dropout(0.1))
    model.add(Dense(1))  # Capa de salida para predecir 'tn'
    model.compile(optimizer='adam', loss='mse')
    return model

# Procesar los datos en lotes para múltiples variables
def process_data_in_batches(scaled_df, n_steps, step_ahead=1):
    models = {}
    predictions = []

    # Agrupar por 'cat1', 'cat2', 'cat3', 'brand', 'customer_id'
    for (cat1, cat2, cat3, brand, customer_id), group_data in scaled_df.groupby(['cat1', 'cat2', 'cat3', 'brand', 'customer_id']):
        group_data = group_data.sort_values(by='periodo')
        
        # Crear secuencias de tiempo
        X, y = crear_secuencias(group_data, n_steps, step_ahead)

        if len(X) == 0 or len(y) == 0:
            continue

        # Escalar los datos de entrada
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X.reshape(-1, 1)).reshape(X.shape)

        # Construir y entrenar el modelo LSTM
        model = build_lstm_model((X_scaled.shape[1], X_scaled.shape[2]))
        model.fit(X_scaled, y, epochs=100, verbose=0)

        # Guardar el modelo
        model_key = (cat1, cat2, cat3, brand, customer_id)
        models[model_key] = model

        # Predecir las ventas para los próximos 2 meses (ejemplo)
        X_pred = group_data[['tn']].values[-(n_steps+step_ahead):-step_ahead].reshape((1, n_steps, 1))
        X_pred_scaled = scaler.transform(X_pred.reshape(-1, 1)).reshape(X_pred.shape)
        pred = model.predict(X_pred_scaled, verbose=0)

        # Desescalar la predicción
        pred_original_scale = scaler.inverse_transform(pred.reshape(-1, 1)).flatten()
        predictions.append([cat1, cat2, cat3, brand, customer_id, pred_original_scale[0]])

    return models, predictions

# Llamada a la función con el DataFrame scaled_df
n_steps = 13
step_ahead = 2
models, predictions = process_data_in_batches(scaled_df, n_steps, step_ahead)



c:\Users\vgARCIARIO\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
c:\Users\vgARCIARIO\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
c:\Users\vgARCIARIO\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs

: 

In [ ]:
# Convertir las predicciones a DataFrame
predictions_df = pd.DataFrame(predictions, columns=['cat1', 'cat2', 'cat3', 'brand', 'customer_id', 'prediction'])

# Sumarizar las predicciones por 'cat1', 'cat2', 'cat3', 'brand'
summarized_preds = predictions_df.groupby(['cat1', 'cat2', 'cat3', 'brand'])['prediction'].sum().reset_index()

# Paso 9: Aplicar los ratios para obtener las predicciones finales por product_id
final_predictions = []
for _, row in summarized_preds.iterrows():
    key = (row['cat1'], row['cat2'], row['cat3'], row['brand'])
    for (cat1, cat2, cat3, brand, product_id), ratio in ratio_dict.items():
        if (cat1, cat2, cat3, brand) == key:
            final_predictions.append([product_id, row['prediction'] * ratio])

# Convertir las predicciones finales a un DataFrame
final_predictions_df = pd.DataFrame(final_predictions, columns=['product_id', 'prediction'])

# Guardar las predicciones finales en un archivo CSV
final_predictions_df.to_csv("C:/Users/vgarciario/desktop/UA MASTER/labo3/predicciones_finales.csv", index=False)

print(final_predictions_df.head())